In [1]:
''' 
¤ in colab:

!pip install optuna
!pip install import_ipynb
!pip install torchmetrics

'''

' \n¤ in colab:\n\n!pip install optuna\n!pip install import_ipynb\n!pip install torchmetrics\n\n'

In [2]:
%reload_ext autoreload
%autoreload 2

import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

import import_ipynb

c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
''' 
¤ in colab:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd '/content/drive/My Drive/Colab Notebooks/IDS_Project'
'''

" \n¤ in colab:\n\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n%cd '/content/drive/My Drive/Colab Notebooks/IDS_Project'\n"

In [4]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from neural_net_model_train import train_net_model, test_net_model

In [6]:
from utils_data_prep_py import get_dataloaders, get_input_size, get_number_of_classes

DEVICE:  cuda
shape of X:  (2827876, 78)
number of labels of y:  15
Class labels:  ['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']


 After spliting the data:
training data shape: (2120907, 23)
test data shape: (706969, 23)
training data shape: (2120907,)
test data shape: (706969,)


c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project\utils_data_prep_py.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X_train = torch.tensor(X_train, dtype = torch.float32).clone().detach()


In [7]:
train_loader,test_loader = get_dataloaders()


In [8]:
input_size = get_input_size()
number_of_classes = get_number_of_classes()

In [9]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight.data,nonlinearity='leaky_relu')
        if m.bias is not None:
          nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)     

In [10]:
# Build a model by implementing define-by-run design from Optuna
def define_model(trial, input_features, numClasses):
    
    no_layers = trial.suggest_int("n_layers", 2, 7)
    layers = []
    num_groups = 4 
    
    layers.append(nn.Flatten())
    for i in range(no_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i), 25, 129)
        
        layers.append(nn.Linear(input_features, out_features)) 
        #num_groups must divide out_features but if out_features is prime than num_groups=1 ,so 1 is good?
        #layers.append(nn.GroupNorm(num_groups, out_features))
        layers.append(nn.BatchNorm1d(out_features))
        layers.append(nn.LeakyReLU())
        drop_procentages = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
        layers.append(nn.Dropout(drop_procentages))
        
        input_features = out_features
        
    layers.append(nn.Linear(input_features, numClasses))
    # layers.append(nn.LogSoftmax(dim=1))# bug: it should be removed next time because CROSSENTROPYLOSS applies LogSoftmax
    
    net_mlp = nn.Sequential(*layers)

    net_mlp.apply(weights_init)

    return nn.Sequential(*layers)

##### Hyperparameter Tuning with Loss Pruning

In [12]:

MAX_EPOCHS = 2

# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

    input_features=input_size
    numClasses=number_of_classes
    #trial controls de parameters of the model
    mlp_model = define_model(trial, input_features, numClasses)
    

    params = {
              'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
    # Generate the optimizers.
    # optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    # lr = trial.suggest_float("lr", 1e-5, 1e-1)
    optimizer = getattr(optim, params['optimizer'])(mlp_model.parameters(), lr=params['learning_rate'])

    
    # Get the FashionMNIST dataset.
    # train_loader, val_loader = get_mnist()

    losses, accuracies = train_net_model(mlp_model, optimizer, train_loader, MAX_EPOCHS, trial_optimisation=trial,trial_parameter='loss')
    accuracy, test_loss = test_net_model(mlp_model, test_loader)

    return accuracy
  
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
# study.optimize(objective, n_trials=30)
study.optimize(objective, n_trials=10)

[I 2022-09-07 17:19:43,072] A new study created in memory with name: no-name-d0cbe08e-5a41-4323-a4ea-7ff92b24d863
C:\Users\androgo\AppData\Local\Temp\ipykernel_9668\3837824746.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
100%|██████████| 65329/65329 [12:11<00:00, 89.27it/s] 


epoch 0 cost 731.854183 sec


100%|██████████| 65329/65329 [10:56<00:00, 99.54it/s] 


epoch 1 cost 656.347196 sec
Finished Training


100%|██████████| 22093/22093 [01:21<00:00, 271.73it/s]
[I 2022-09-07 17:44:12,857] Trial 0 finished with value: 0.8031893968582153 and parameters: {'n_layers': 3, 'n_units_l0': 80, 'dropout_l0': 0.35240660565264426, 'n_units_l1': 44, 'dropout_l1': 0.29170417780502633, 'n_units_l2': 44, 'dropout_l2': 0.25661269918569585, 'learning_rate': 0.0207145198198421, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.8031893968582153.



Test set: Average loss: 1.7559, Accuracy: 0/706969 (1%)



100%|██████████| 65329/65329 [10:24<00:00, 104.60it/s]


epoch 0 cost 624.620018 sec


 68%|██████▊   | 44365/65329 [06:29<02:48, 124.21it/s]

In [ ]:
from mlp.mlp_model_train import train_MLP_Model, test_mlp


In [ ]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

n_layers: 7
n_units_l0: 79
dropout_l0: 0.3106931205219248
n_units_l1: 52
dropout_l1: 0.3442582466133433
n_units_l2: 125
dropout_l2: 0.4707430303682435
n_units_l3: 106
dropout_l3: 0.3339376061596629
n_units_l4: 86
dropout_l4: 0.48873164723845
n_units_l5: 99
dropout_l5: 0.2888867540062092
n_units_l6: 81
dropout_l6: 0.3829947185640129
learning_rate: 0.00014512993424558004
optimizer: RMSprop


# Tuning Visualization
###### graphs below are not visible in vs code but are in colab 

In [ ]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

##### Hyperparameter Tuning with Accuracy Pruning

In [ ]:
from mlp.mlp_model_train import train_MLP_Model, test_mlp

MAX_EPOCHS = 7

# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

    input_features=input_size
    numClasses=number_of_classes
    #trial controls de parameters of the model
    mlp_model = define_model(trial, input_features, numClasses)
    

    params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
    # Generate the optimizers.
    # optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    # lr = trial.suggest_float("lr", 1e-5, 1e-1)
    optimizer = getattr(optim, params['optimizer'])(mlp_model.parameters(), lr=params['learning_rate'])

    
    # Get the FashionMNIST dataset.
    # train_loader, val_loader = get_mnist()

    losses, accuracies = train_MLP_Model(mlp_model, optimizer, train_loader, MAX_EPOCHS, trial_optimisation=trial,trial_parameter='accuracy')
    accuracy = test_mlp(mlp_model, test_loader)

    return accuracy
  
  
#EPOCHS = 30
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30)
# study.optimize(objective, n_trials=10)

100%|██████████| 65329/65329 [15:50<00:00, 68.72it/s]


epoch 6 cost 950.671599 sec
Finished Training


100%|██████████| 22093/22093 [01:52<00:00, 195.97it/s]
[I 2022-09-07 09:21:07,668] Trial 3 finished with value: 0.8031893968582153 and parameters: {'n_layers': 5, 'n_units_l0': 71, 'dropout_l0': 0.2627462467563766, 'n_units_l1': 99, 'dropout_l1': 0.3101353981503837, 'n_units_l2': 90, 'dropout_l2': 0.46803509627894024, 'n_units_l3': 86, 'dropout_l3': 0.31380410843800677, 'n_units_l4': 120, 'dropout_l4': 0.4776293397546746, 'learning_rate': 0.033010075551570366, 'optimizer': 'Adam'}. Best is trial 3 with value: 0.8031893968582153.



Test set: Average loss: 1.3104, Accuracy: 0/706969 (1%)



100%|██████████| 65329/65329 [06:00<00:00, 181.23it/s]


epoch 0 cost 360.479229 sec


 38%|███▊      | 24767/65329 [02:21<03:51, 175.50it/s]
[W 2022-09-07 09:29:29,328] Trial 4 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\androgo\AppData\Local\Temp\ipykernel_23692\3833815060.py", line 28, in objective
    losses, accuracies = train_MLP_Model(mlp_model, optimizer, train_loader, MAX_EPOCHS, trial_optimisation=trial,trial_parameter='accuracy')
  File "c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project\mlp\mlp_model_train.py", line 193, in train_MLP_Model
    accuracy.update(outputs,batch_y)
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\metric.py", line 391, in wrapped_func
    update(*args, **kwargs)
  File "c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\torchmetrics\classific

KeyboardInterrupt: 

In [ ]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

n_layers: 5
n_units_l0: 71
dropout_l0: 0.2627462467563766
n_units_l1: 99
dropout_l1: 0.3101353981503837
n_units_l2: 90
dropout_l2: 0.46803509627894024
n_units_l3: 86
dropout_l3: 0.31380410843800677
n_units_l4: 120
dropout_l4: 0.4776293397546746
learning_rate: 0.033010075551570366
optimizer: Adam


In [ ]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)